In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import copy


In [18]:
# dataset1 = pd.read_csv("data1.txt", delimiter = "\t", skiprows = [0,0])
dataset2 = pd.read_csv("data2.txt", delimiter = "\t")
dataset3 = pd.read_csv("data3.txt", delimiter = "\t")

In [19]:
dataset1 = pd.read_csv("data1.txt", delim_whitespace=True, dtype=str) # 32 rows x 5 variables (+ class, space separated, CR EOL)

In [20]:
dataset1.loc[0][0]

'00000'

In [21]:
np_data_set = []


for item in dataset1.itertuples():
    bit_string = str(item[1])
    output_string = str(item[2])
    np_bit_string = np.fromstring(bit_string,'u1') - ord('0')
    np_output_string = np.fromstring(output_string,'u1') - ord('0')
    # print(np_bit_string, np_output_string)
    np_data_set.append([np_bit_string, np_output_string])

<ipython-input-21-8fabb802a3bc>:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_bit_string = np.fromstring(bit_string,'u1') - ord('0')
<ipython-input-21-8fabb802a3bc>:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_output_string = np.fromstring(output_string,'u1') - ord('0')


In [22]:
def schema_model(schema_bitstring_array: 'np.ndarray', schema_target: 'np.ndarray', test_data_array: 'np.ndarray', test_target_result: 'np.ndarray') -> bool:
    """
    This function handles np.ndarray of 1D, lists, that is to say.
    : schema is the mask
    : test is data from the dataset
    """

    # we take the schmea (the values that are 1), and take values as per it from the test_data_array
    # 0 to 5 is the schema (the 5th index is actually the 6th value, but numpy doesnt touch the last value. but only the value before that)
    # ie: we use the schmea as a mask/ filter over the test data and extract those values (which are zeros or ones)

    # print(schema_bitstring_array, "schema_bitstring_array")
    # print(schema_bitstring_array[0:5], "schema_bitstring_array[0:5]")
    # print(test_data_array, "test_data_array")
    data_array = test_data_array[schema_bitstring_array == 1]
    # print(data_array, "filtered data")
    
    # we use the sum the filtered data into an integer
    sum_of_data_array = data_array.sum()
    # print(sum_of_data_array)

    # this is hypothesized output from the schema
    schema_target_output = sum_of_data_array >= schema_target[0]

    # compare against the actual output
    actual_output = None 
    
    if test_target_result[0] == 0:
        actual_output = False
    elif test_target_result[0] == 1:
        actual_output = True
    
    does_both_match = actual_output and schema_target_output

    # print(actual_output, schema_target_output, does_both_match)
    return does_both_match



# first 5 values have to be bool values, the 6th value has to be a value between 1 and 5
random_matrix_array = np.random.randint(2,size=(1,5))
random_matrix_array_2 = np.random.randint(2,size=(1,5))
random_target = np.random.choice([0, 1], size=(1, 1)) # actual target output
random_target_t = np.random.choice([0, 1, 2, 3, 4], size=(1, 1)) # hypothesized random output // np_data_set


# print("random schema", random_matrix_array[0], "random sum threhold", random_target_t[0])
# print("actual data  ", random_matrix_array_2[0], "output             ", random_target[0])
# schema_model(schema_bitstring_array=random_matrix_array[0], schema_target=random_target_t[0], test_data_array=random_matrix_array_2[0], test_target_result=random_target[0])

random_int = random.randrange(0, 30, 1)

# print("random schema", random_matrix_array[0], "random sum threhold", random_target_t[0])
# print("actual data  ", np_data_set[random_int][0], "output             ", np_data_set[random_int][1])

schema_model(schema_bitstring_array=random_matrix_array[0], schema_target=random_target_t[0], test_data_array=np_data_set[random_int][0], test_target_result=np_data_set[random_int][1])

False

In [36]:
def schema_model_v2(schema_bitstring_array: 'np.ndarray', schema_target: 'np.ndarray', schema_value_map: 'np.ndarray', test_data_array: 'np.ndarray', test_target_result: 'np.ndarray') -> bool:
    """
    This function handles np.ndarray of 1D, lists, that is to say.
    : schema is the mask
    : test is data from the dataset
    """
    # print(test_data_array, " ", schema_value_map)

    data_array = test_data_array[schema_bitstring_array == 1]
    value_array = schema_value_map[schema_bitstring_array == 1]
    mult_array = np.matmul(data_array, value_array)

    # print(data_array, "filtered data")
    
    # we use the sum the filtered data into an integer
    sum_of_data_array = mult_array.sum()
    # print(sum_of_data_array)

    # this is hypothesized output from the schema
    schema_target_output = sum_of_data_array >= schema_target[0]

    # compare against the actual output
    actual_output = None 
    
    if test_target_result[0] == 0:
        actual_output = False
    elif test_target_result[0] == 1:
        actual_output = True
    
    does_both_match = actual_output and schema_target_output

    # print(actual_output, schema_target_output, does_both_match)
    return does_both_match

#####################################################################

class Chromosome:
    def __init__(self):
        # the schema for fitness function is the genome, it is an array of [[Bool, Bool, Bool, Bool, Bool], [1 to 5]]
        self.dna_schema = []
        self.dna_schema_value_map = []
        self.dna_target = []
        self.fitness = 0.0

    def initialize(self) -> None:
        random_matrix_array = np.random.randint(2,size=(1,5))
        self.dna_schema = random_matrix_array[0]


        random_matrix_array_2 = np.random.randint(100,size=(1,5))
        self.dna_schema_value_map = random_matrix_array_2[0]

        min_target = np.amin(random_matrix_array_2)
        max_target = random.randint(min_target, random_matrix_array_2.sum())
        np_random_target_2 = np.array([max_target])
        self.dna_target = np_random_target_2

    def initialize_with_values(self, _dna_schema, _dna_schema_value_map) -> None:
        self.dna_schema = _dna_schema
        self.dna_schema_value_map = _dna_schema_value_map
        self.dna_target = []
        self.fitness = 0.0

        min_target = np.amin(self.dna_schema_value_map)
        max_target = random.randint(min_target, self.dna_schema_value_map.sum())
        np_random_target_2 = np.array([max_target])
        self.dna_target = np_random_target_2


    def calculate_fitness(self, actual_dataset, fitness_function) -> None:

        for actual_data_string, actual_result in actual_dataset:
            is_match = fitness_function(schema_bitstring_array=self.dna_schema, 
                                        schema_target=self.dna_target, 
                                        schema_value_map=self.dna_schema_value_map, 
                                        test_data_array=actual_data_string, 
                                        test_target_result=actual_result)
            
            if is_match:
                self.fitness+=1

        # print(self.fitness, self.dna_schema, self.dna_schema_value_map, self.dna_target)

    def cross_over(self, mate_chromosome) -> list:
        this_chromosome_schema_split = np.array_split(self.dna_schema, 2)
        mate_chromosome_schema_split = np.array_split(mate_chromosome.dna_schema, 2)
        child_a_schema = np.concatenate((this_chromosome_schema_split[0], mate_chromosome_schema_split[1]))
        child_b_schema = np.concatenate((mate_chromosome_schema_split[0], this_chromosome_schema_split[1]))
        clone_schema = copy.deepcopy(self.dna_schema)

        this_chromosome_value_map_split = np.array_split(self.dna_schema_value_map, 2)
        mate_chromosome_value_map_split = np.array_split(mate_chromosome.dna_schema_value_map, 2)
        child_a_value_map = np.concatenate((this_chromosome_value_map_split[0], mate_chromosome_value_map_split[1]))
        child_b_value_map = np.concatenate((mate_chromosome_value_map_split[0], this_chromosome_value_map_split[1]))
        clone_value_map = copy.deepcopy(self.dna_schema_value_map)

        child_a = Chromosome()
        child_a.initialize_with_values(child_a_schema, child_a_value_map)

        child_b = Chromosome()
        child_b.initialize_with_values(child_b_schema, child_b_value_map)

        child_clone = Chromosome()
        child_clone.initialize_with_values(clone_schema, clone_value_map)

        return [child_a, child_b, child_clone]
        

    def mutate_schema(self, _mutation_rate: int) -> None:
        _mutation_rate_round_upped = np.ceil(_mutation_rate)
        for count, gene in enumerate(self.dna_schema):
            random_int = random.randint(0, 100)
            if random_int <= _mutation_rate_round_upped:
                gene_inversed = np.where((gene==0)|(gene==1), gene^1, gene) # XOR operator
                self.dna_schema[count] = gene_inversed

    def mutate_value_map(self, _mutation_rate: int) -> None:
        _mutation_rate_round_upped = np.ceil(_mutation_rate)
        for count, gene in enumerate(self.dna_schema_value_map):
            random_int = random.randint(0, 100)
            if random_int <= _mutation_rate_round_upped:
                random_arr = np.random.randint(100,size=(1,1))
                random_value = random_arr[0][0]
                self.dna_schema_value_map[count] = random_value

                min_target = np.amin(random_arr)
                max_target = random.randint(min_target, random_arr.sum())
                np_random_target = np.array([max_target])
                self.dna_target = np_random_target

    def __str__(self):
        return f"schema: {self.dna_schema}, target: {self.dna_target}, value_map: { self.dna_schema_value_map }"

    
    def __repr__(self):
        return f"schema: {self.dna_schema}, target: {self.dna_target}, value_map: { self.dna_schema_value_map }"


#####################################################################

class Popultion:
    def __init__(self, _mutation_rate, _population_size, _max_generations):
        self.mutation_rate = _mutation_rate
        self.population_size = _population_size
        self.generations = _max_generations

        self.population_list = []
        self.mating_pool = []
        self.best_Chromosome = Chromosome()
        self.fitness_records = []




x = Chromosome()
x.initialize()
print(x)
x.mutate_value_map(2.1)
print(x)

schema: [1 1 1 0 1], target: [170], value_map: [69 54 10 20 97]
schema: [1 1 1 0 1], target: [3], value_map: [ 3 54 10 20 97]
